In [1]:
import time
import json

import gensim
from elasticsearch import Elasticsearch

c:\miniconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# client = Elasticsearch("localhost:9200")
client = Elasticsearch("localhost:9292")

In [3]:
print('Start loading...')
start = time.time()
model = gensim.models.KeyedVectors.load_word2vec_format("../data/glove.6B.50d.txt")
print('Load model time consumed: {:.2f}'.format(time.time() - start))

Start loading...


c:\miniconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Load model time consumed: 23.28


In [4]:
def query(word, cat=None, index_name='word_vector_index', size=3):
    query_vector = model.get_vector(word).tolist()
    if cat is None:
        script_query = {
            "query": {
                "script_score": {
                    "query": {
                        "match_all": {}
                    },
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, doc['word_vector']) + 1.0",
                        "params": {
                            "query_vector": query_vector
                        }
                    }
                }
            },
            "_source": ["word"]
        }
    else:
        script_query = {
            "query": {
                "script_score": {
                    "query": {
                        "match": {
                            "category": cat
                        }
                    },
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, doc['word_vector']) + 1.0",
                        "params": {
                            "query_vector": query_vector
                        }
                    }
                }
            },
            "_source": ["word"]
        }
    response = client.search(
        index=index_name,
        body=script_query,
        size=size
    )
    print(json.dumps(response['hits']['hits'], indent=4))

In [5]:
query("mice", 'sports')

[
    {
        "_index": "word_vector_index",
        "_type": "_doc",
        "_id": "2",
        "_score": 1.2059406,
        "_source": {
            "word": "swimming"
        }
    },
    {
        "_index": "word_vector_index",
        "_type": "_doc",
        "_id": "0",
        "_score": 1.1205508,
        "_source": {
            "word": "basketball"
        }
    },
    {
        "_index": "word_vector_index",
        "_type": "_doc",
        "_id": "4",
        "_score": 1.1117831,
        "_source": {
            "word": "run"
        }
    }
]


In [6]:
query("mice", 'food')

[
    {
        "_index": "word_vector_index",
        "_type": "_doc",
        "_id": "8",
        "_score": 1.4371431,
        "_source": {
            "word": "milk"
        }
    },
    {
        "_index": "word_vector_index",
        "_type": "_doc",
        "_id": "5",
        "_score": 1.1839651,
        "_source": {
            "word": "cheese"
        }
    },
    {
        "_index": "word_vector_index",
        "_type": "_doc",
        "_id": "7",
        "_score": 1.1520243,
        "_source": {
            "word": "beer"
        }
    }
]
